# Продвинутый Python, семинар 10

**Лектор:** Петров Тимур

**Семинаристы:** Петров Тимур, Коган Александра, Бузаев Федор, Дешеулин Олег

**Spoiler Alert:** в рамках курса нельзя изучить ни одну из тем от и до досконально (к сожалению, на это требуется больше времени, чем даже 3 часа в неделю). Но мы попробуем рассказать столько, сколько возможно :)

## Docker

### Но... зачем?

Теперь мы перешли к самой сложной части - про докер.

Что такое докер? Представьте себе ситуацию: у вас есть приложение, которое вы хотите запустить и показать всему миру, какие вы классные. Что делать?

Можно, конечно, создать виртуалку, на нее накинуть приложение и там все делать. Но в какой-то момент вам придется класть туда несколько приложений и дальше начинается катавасия с загруженностью процесса. В какой-то момент одно приложение может стрельнуть (например, если у вас отдельное приложение для новостей, куда зайдет сразу дофига людей), сожрет все процессы и остальные части будут страдать. А еще же люди заходят с разными ОС, и надо конфигурации разные иметь, чтобы у всех все работало. Вот для этих вещей и есть контейнеры

Что такое контейнер? Это способ стандартизации развертки приложения и отделения его от общей инфраструктуры. Экземпляр приложения запускается в изолированной среде, не влияющей на основную операционную систему


![](https://d1.awsstatic.com/Developer%20Marketing/containers/monolith_2-VM-vs-Containers.78f841efba175556d82f64d1779eb8b725de398d.png)

Контейнер позволяют:

1. упаковать в единый образ приложение и все его зависимости: библиотеки, системные утилиты и файлы настройки. Это упрощает перенос приложения на другую инфраструктуру

2. приложения работают только внутри контейнеров и не имеют доступа к основной операционной системе. Это повышает безопасность приложений:они не смогут случайно или умышленно навредить основной системе. Если приложение в контейнере завершится с ошибкой или зависнет, это никак не затронет основную ОС (изоляция ресурсов)

3. избавляет от зависимости ОС: достаточно добавить необходимую конфигурацию в контейнер вместо процесса эмулирования одной ОС на другой (что трудозатратно)

4. За счет оптимизации контейнеров получаем также меньшую загрузку

### Ну что же, начнем

Первое, что надо сделать - это загрузить образ из Docker Hub. Образ (Image) - это схема нашего приложения, основа контейнера, с помощью которого его можно запустить. Все возможные образы хранятся [здесь](https://hub.docker.com/search?q=&type=image) (можно будет добавить сюда же и свои образы, которые вам необходимы)

Загрузим самый простой образ - busybox (дальше будет работа в терминале, а не в Колабе)

In [ ]:
docker pull busybox # загрузи образ busybox, если Permission denied, то запустите с sudo
docker images # посмотреть на все загруженные образы
docker rmi busybox # удалить образ

Отлично, загрузили, давайте запускать!

In [ ]:
docker run busybox # ничего не случилось, потому что мы ничего и не задали
docker run busybox echo "Hello" # о, что-то выдал
docker ps # посмотреть запущенные контейнеры (их пока нет, потому что прошлые кончились)
docker ps -a # посмотреть все контейнеры
docker run -it busybox sh #запустить на больше, чем 1 команду (-it - флаг интерактива)

Все запушенные и кончившиеся контейнеры занимают место, поэтому их еще стоит удалять (как минимум те, которые завершили свое действие)

In [ ]:
docker rm <ids> # удалить все ненужные контейнеры
docker rm $(docker ps -a -q -f status=exited) # чтобы не копировать все id
# -a все контейнеры, -q  - вывести только ID, -f - фильтр (на статус закончившися exited)

Обратите внимание сверху на значок Docker (MacOS). Видим, что Docker работает. Что это значит? Что у нас запушен процесс, с помощью которого это все вообще происходит (так называемый Docker Daemon)

### Запустим сайт из под капота

Окей, повеселились с каким-то образом. Как приложения-то запускать? Давайте попробуем запустить какой-нибудь более интересный образ

In [ ]:
docker pull prakhar1989/static-site
docker run prakhar1989/static-site #ничего не происходит, видим просто что is running
docker stop $(docker ps -a -q -f status=running) #выключаем все активные контейнеры
docker run -d -P --name static-site prakhar1989/static-site #давайте сделаем так:
# -d - открепляем наш терминал от контейнера(не будет прикреплен, можем продолжить работу в терминале)
# -P - сделаем порты открытыми и публичными, чтобы подключиться
# --name переименуем для удоства в static-site
docker port static-site #смотрим на порты, открываем на localhost:<порт>

Ура, запустили простенький сайт! А если поменять порт? Можно

In [ ]:
docker run -d -P -p 8888:80 prakhar1989/static-site --name static-site prakhar1989/static-site #теперь он на порту 8888
# окей, остановим все и удалим

### Хотим быть крутыми со своим образом!

Теперь хотим создать свой собственный образ и сделать еще интереснее. Образы делятся на 2 типа:

* Официальные образы - поддерживаются командой docker, скачать можно по названию (например, docker pull python)

* Неофициальные образы - образ, созданный пользователем, чаще всего выглядит как user/name

Скачаем простенький сайт, сделанный на Flask (о нем мы будет говорить в других семинарах) и запустим его локально

In [ ]:
git clone https://github.com/prakhar1989/docker-curriculum.git #скачаем для простого приложения на Flask (о нем мы будет говорить в других семинарах)
cd docker-curriculum/flask-app; pip install -r requirements.txt #установим все, что нужно для сайта
vim app.py #для того, чтобы у меня запустилась, так как порт 5000 у меня занят, я поменял в коде на 8888
python app.py

Получилось! Теперь создадим образ с этим приложением. Что нужно?

Так как сайт написан на Python, то надо скачать базовый образ python

In [ ]:
docker pull python:3-onbuild #Какой onbuild, це шо? Это надстройка при запуске возьмет на requirements.txt и установит за нас все, что нужно
# То есть такой помощник при запуске

Теперь надо как-то соединить приложение и образ. Это делается с помощью DockerFile - текстовый документ для автоматизации сборки (код для DockerFile почти идентичен аналогам bash)

Создаем там же, где и наше приложение файл Dockerfile и прописываем:

In [ ]:
vim Dockerfile

In [ ]:
FROM python:3-onbuild #указываем образ, который надо использовать

#благодаря тому, что у нас onbuild, нам не надо копировать файлы и устанавливать зависимости здесь

EXPOSE 8888 #говорим, на какой порт это все отправлять

CMD ["python", "./app.py"] # команды для запуска (то есть что надо сделать)

Осталось собрать образ через docker build:

In [ ]:
docker build -t palladain7/catgif . #здесь надо зарегаться на Docker Hub (это быстро) и в качестве user ввести свой ник
docker run -p 8888:8888 palladain/catgif #собрали-запустили
docker images #проверяем образ

Ура, мы собрали образ докера! Теперь осталось его загрузить на Docker Hub, чтобы его могли увидеть все и использовать)

In [ ]:
docker login #вначале надо авторизоваться
docker push palladain7/catgif #пушим
https://hub.docker.com/r/palladain7/catgif/ - проверяем, успех

## А теперь со всей этой информации можно уже и бота загрузить на docker!

### Делаем бота сильным и независимым

In [ ]:
%%writefile bot.py

from random import seed, randrange
from time import time
import telebot
from telebot import types

TOKEN = '5674479560:AAHI0lWyLHZQUa91Di-6NmNqdWbE7lL_6H8' # указываем токен нашего бота (для этого надо создать бота в @BotFather)
# Создайте собственного бота, чтобы наши наработки друг друга не перебивали

bot = telebot.TeleBot(TOKEN) # инициализируем нашего бота

parrots = {1: 'https://cindygurmann.files.wordpress.com/2018/06/ea2530ad-e913-4d5b-8036-762b5b227c04.jpeg',
           2: 'https://cherepah.ru/wp-content/uploads/2/2/8/228937ec782b8755993a3241e1d6c039.jpeg',
           3: 'https://kotsobaka.com/wp-content/uploads/2018/08/2748131046_8a253489b5_b.jpg',
           4: 'https://bestpopugai.ru/wp-content/uploads/2022/05/1-5.jpg',
           5: 'https://i.artfile.ru/1920x1200_952300_[www.ArtFile.ru].jpg',
           6: 'https://kipmu.ru/wp-content/uploads/pchppgks.jpg'}

favourite_parrot = 'https://pet7.ru/wp-content/uploads/2017/09/Popugaj-zhako-osobennosti-vida.jpg'

parrot_gif = 'https://i1.wp.com/cdn.dribbble.com/users/104127/screenshots/2589080/parrots.gif'

@bot.message_handler(commands=['start'])
def hello_message(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2) # указываем, сколько кнопок может быть в строке
    item_1 = types.KeyboardButton("Любимый попугай")
    item_2 = types.KeyboardButton("🎲")
    markup.add(item_1, item_2) #добавляем
    bot.send_message(message.chat.id, "Привет, тут будут попугаи!", reply_markup=markup)

@bot.message_handler(content_types=['text', 'emoji'])
def message_reply(message):
    if message.text=="Любимый попугай":
        markup = types.ReplyKeyboardMarkup()
        item = types.KeyboardButton(text='Хочу GIF-ку!')
        markup.add(item)
        bot.send_photo(message.chat.id, favourite_parrot, reply_markup = markup)
    elif message.text == "🎲":
        r = bot.send_dice(message.chat.id)
        bot.send_photo(message.chat.id, parrots[r.dice.value])
    elif message.text == 'Хочу GIF-ку!':
        markup = types.InlineKeyboardMarkup()
        item = types.InlineKeyboardButton(text='Ссылка', url=parrot_gif)
        item_1 = types.InlineKeyboardButton(text='Переслать', switch_inline_query=parrot_gif)
        markup.add(item, item_1)
        bot.send_animation(message.chat.id, parrot_gif, reply_markup = markup)
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2) # указываем, сколько кнопок может быть в строке
        item_1 = types.KeyboardButton("Любимый попугай")
        item_2 = types.KeyboardButton("🎲")
        markup.add(item_1, item_2) #добавляем
        bot.send_message(message.chat.id, "Еще попуги!", reply_markup = markup)

@bot.inline_handler(func=lambda query: len(query.query) > 0)
def query_text(query):
    try:
        seed(int(time()))
        size = randrange(1, 100)
        print(size)
        r_sum = types.InlineQueryResultArticle(
                id='1', title="Папуг!",
                input_message_content = types.InputTextMessageContent(message_text="Ваш папуг " + str(size) + " размера" )
        )
        bot.answer_inline_query(query.id, [r_sum])
    except Exception as e:
        print("{!s}\n{!s}".format(type(e), str(e)))

bot.polling(none_stop=True, interval=0) #запускаем нашего бота

Writing bot.py


Вспоминаем, как работать с докером:

In [ ]:
docker pull python:3-onbuild

vim Dockerfile

In [ ]:
FROM python:3-onbuild #указываем образ, который надо использовать

#благодаря тому, что у нас onbuild, нам не надо копировать файлы и устанавливать зависимости здесь

EXPOSE 8888 #говорим, на какой порт это все отправлять

CMD ["python", "bot.py"] # команды для запуска (то есть что надо сделать)

In [ ]:
docker build -t palladain7/catgif . #здесь надо зарегаться на Docker Hub (это быстро) и в качестве user ввести свой ник
docker run -d -P -p 8888:8888 palladain/catgif #собрали-запустили
docker images #проверяем образ

## XML

BS также умеет парсить XML, но что это такое?

XML (eXtensible Markup Language) - так называемый расширяемый язык разметки. Выглядит он примерно так же, как и HTML, только тэги не являются командами, по существу. Давайте разберем на примере:

In [ ]:
!cat external-40779.xml

Что видим? Тэг открытия-закрытия, и тэги, внутри которых лежат параметры. А также внутри тэгов могут лежать тэги или текст. В целом, почти как HTML

Для обработки XML есть отдельная [библиотека](https://lxml.de/)

In [ ]:
!pip install lxml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from lxml import etree

tree = etree.parse('external-40779.xml')
root = tree.getroot()
print(root.tag) #название тэга
print(root.text) #текст внутри тэга

contestLog

  


In [ ]:
for elem in root.findall("users/user"): # поиск по вложенности <users> <user>
    print(elem.attrib["id"]) # обращение к атрибутам и к конкретному атрибуту

Так вот, парсеры внутри BS берутся из этой библиотеки (так что можно просто использовать BS)

## JSON

С JSON мы один раз работали на семинаре, но давайте повторим-пройдем еще раз, что это такое и с чем его едят

JSON - JavaScript Object Notation (формат, унаследованный от JavaScript, но сейчас живет независимо). Он еще более читабельный, чем XML, и очень часто используется в REST API, поэтому надо знать врага в лицо

Как он [выглядит](https://www.json.org/json-en.html):

```
{
   "firstName": "Иван",
   "lastName": "Иванов",
   "address": {
       "streetAddress": "Московское ш., 101, кв.101",
       "city": "Ленинград",
       "postalCode": 101101
   },
   "phoneNumbers": [
       "812 123-1234",
       "916 123-4567"
   ]
}
```

Почти словарь. У любого атрибута должно быть название, значения идут после двоеточия. Могут быть вложены как словари, может быть список

И всегда двойные кавычки! Давайте запишем JSON в файл

In [ ]:
%%writefile example.json

{
   "firstName": "Иван",
   "lastName": "Иванов",
   "address": {
       "streetAddress": "Московское ш., 101, кв.101",
       "city": "Ленинград",
       "postalCode": 101101
   },
   "phoneNumbers": [
       "812 123-1234",
       "916 123-4567"
   ]
}

Writing example.json


Внутри Python есть встроенная библиотека для обработки, называется (json)[https://docs.python.org/3/library/json.html] (кто бы мог подумать)

Библиотека маленькая и имеет 4 метода:

* load - считываем из файла данные

* loads - считываем данные из s (string или байты)

* dump - загружаем в файл

* dumps - загружаем в строку

In [ ]:
import json

with open("example.json", "r") as f:
    r = json.load(f) #Считать json
print(r)
type(r)

{'firstName': 'Иван', 'lastName': 'Иванов', 'address': {'streetAddress': 'Московское ш., 101, кв.101', 'city': 'Ленинград', 'postalCode': 101101}, 'phoneNumbers': ['812 123-1234', '916 123-4567']}


dict

In [ ]:
with open("example_1.json", "w") as f:
    json.dump(r, f) #Грузим json

!cat example_1.json # Ой как плохо, будет нечитабельно, что же делать?

{"firstName": "\u0418\u0432\u0430\u043d", "lastName": "\u0418\u0432\u0430\u043d\u043e\u0432", "address": {"streetAddress": "\u041c\u043e\u0441\u043a\u043e\u0432\u0441\u043a\u043e\u0435 \u0448., 101, \u043a\u0432.101", "city": "\u041b\u0435\u043d\u0438\u043d\u0433\u0440\u0430\u0434", "postalCode": 101101}, "phoneNumbers": ["812 123-1234", "916 123-4567"]}

In [ ]:
with open("example_1.json", "w") as f:
    json.dump(r, f, ensure_ascii=False) #Грузим json, говорим, что там не ascii

!cat example_1.json

{"firstName": "Иван", "lastName": "Иванов", "address": {"streetAddress": "Московское ш., 101, кв.101", "city": "Ленинград", "postalCode": 101101}, "phoneNumbers": ["812 123-1234", "916 123-4567"]}

Что можно запихнуть в JSON из Python?

* dict
* list
* tuple
* string
* int
* float
* True
* False
* None

Остальное не поймет :с

In [ ]:
print(r["firstName"])
print(r["phoneNumbers"])

Иван
['812 123-1234', '916 123-4567']


Можно загрузать сразу несколько json из файла (но только если они в формате JSON lines)

In [ ]:
%%writefile example.json
{"firstName": "Иван","lastName": "Иванов","streetAddress": "Московское ш., 101, кв.101"}
{"firstName": "Иван","lastName": "Иванов","streetAddress": "Московское ш., 101, кв.101"}
{"firstName": "Иван","lastName": "Иванов","streetAddress": "Московское ш., 101, кв.101"}

Overwriting example.json


In [ ]:
data = []
with open("example.json", "r") as f:
    for line in f:
        data.append(json.loads(line)) #Считать json
data

[{'firstName': 'Иван',
  'lastName': 'Иванов',
  'streetAddress': 'Московское ш., 101, кв.101'},
 {'firstName': 'Иван',
  'lastName': 'Иванов',
  'streetAddress': 'Московское ш., 101, кв.101'},
 {'firstName': 'Иван',
  'lastName': 'Иванов',
  'streetAddress': 'Московское ш., 101, кв.101'}]

Если несколько JSONов записано не так, то это считается плохим файлом, так что :с

Можно попробовать руками обрабатывать, конечно...

# И немного [заданий](https://mariolurig.com/crossword/) на регулярки!

### Задание 1

На вход даётся текст. Необходимо вывести все e-mail адреса, которые в нём встречаются. В общем виде задача достаточно сложная, поэтому у нас будет 3 ограничения:

* две точки внутри адреса не встречаются;
* две собаки внутри адреса не встречаются;
* считаем, что e-mail может быть частью «слова», то есть в boo@ya_ru мы видим адрес boo@ya, а в foo№boo@ya.ru видим boo@ya.ru

### Задание 2

Будем считать аббревиатурой слова только лишь из заглавных букв (как минимум из двух). Если несколько таких слов разделены пробелами, то они
считаются одной аббревиатурой. Найдите все возможные аббревиатуры в тексте

### Задание 3

На вход даётся текст. Выведите слитно первые буквы каждого слова. Буквы необходимо выводить заглавными.
Эту задачу можно решить в одну строчку.

### Задание 4

Довольно распространённая ошибка ошибка — это повтор слова.
Вот в предыдущем предложении такая допущена. Необходимо исправить каждый такой повтор (слово, один или несколько пробельных символов, и снова то же слово).

### Задание 5

На вход даётся текст. Необходимо вывести все e-mail адреса, которые в нём встречаются. При этом e-mail не может быть частью слова, то есть слева и справа от e-mail'а должен быть либо конец строки, либо не-буква и при этом не один из символов '._+-, допустимых в адресе.

### Задание 6 (на BS)

Дан сайт http://mignews.com/mobile (сайт с новостями)

Выведите оттуда все новости в ленты новостей и выведите их, а также найдите все тэги с буквой k и вы ведите их значения)